<a href="https://colab.research.google.com/github/coolShub/DiceGame/blob/main/18bce1006and18bce1040(FAQs_Question_Answering_Chatbot_Interaction_Using_Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("FAQ_MachineLearningInterview_com.csv", delimiter='|')

df

In [ ]:

import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence , stopwords=False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
        sentence = remove_stopwords(sentence)

    #sent_stemmed=''
    #for word in sentence.split():
    #    sent_stemmed += ' ' + st.stem(word)
    #sentence = sent_stemmed
    return sentence

def get_cleaned_sentences(df,stopwords=False):
    sents=df[["questions"]];
    cleaned_sentences=[]
    
    for index,row in df.iterrows():
        #print(index,row)
        cleaned = clean_sentence(row["questions"],stopwords);
        cleaned_sentences.append(cleaned)

    return cleaned_sentences;

cleaned_sentences = get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print('\n')

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

In [ ]:
import numpy

sentences = cleaned_sentences_with_stopwords
#sentences = cleaned_sentences

sentence_words = [[word for word in document.split()] for document in sentences]
print(sentence_words)

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)

for key,value in dictionary.items():
    print(key, " : ", value)

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]

for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

#question_orig="do I need to learn algorithms to be a data scientist ?"
question_orig="What does a data scientist usually do ?"
# question_orig=input("Enter the query")
question=clean_sentence(question_orig,stopwords=False)
question_embedding=dictionary.doc2bow(question.split())

print("\n\n",question,"\n",question_embedding)


In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    # max_sim=-1
    # index_sim=-1

    total_lst = []
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1,-1),question_embedding.reshape(1,-1))[0][0]
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
        print(index,sim,sentences[index])
        total_lst.append((sim, index))
        # if sim > max_sim:
        #     max_sim = sim
        #     index_sim = index
    
    total_lst.sort(reverse=True)

    print("\n")
    print("Question: ",question)

    # print("index             sim                   ques")
    for siml, index in total_lst:
        if siml == total_lst[0][0]:
            print("\n")
            # print("\n")
            print("Retrieved: ",FAQdf.iloc[index,0])
            print(FAQdf.iloc[index,1])
            # print(index, "               ", siml, "               ", sentences[index])
    # print(total_lst)


retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences)


In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

glove_model=None

try:
    glove_model=gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")



w2v_model=None

try:
    w2v_model=gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:
    w2v_model = api.load('word2vec-google-news-300')
    w2v_model.save("./w2vecmodel.mod")
    print("Saved w2v model")


w2vec_embedding_size=len(w2v_model['computer'])
glove_embedding_size=len(glove_model['computer'])

In [ ]:
def getWordVec(word, model):
	samp = model['computer']
	vec = [0]*len(samp)
	try:
		vec = model[word]
	except:
		vec = [0]*len(samp)
	return (vec)
	
def getPhraseEmbedding(phrase, embeddingmodel):
	samp = getWordVec('computer', embeddingmodel)
	vec = numpy.array([0]*len(samp))
	den = 0
	for word in phrase.split():
		print(word)
		den = den + 1
		vec = vec + numpy.array(getWordVec(word, embeddingmodel))
		
	#vec  vec/den;
	#return (vec.tolist())
	return vec.reshape(1, -1)

In [ ]:
#With w2Vec

sent_embeddings = []
for sent in cleaned_sentences:
	sent_embeddings.append(getPhraseEmbedding(sent, w2v_model))
	
question_embedding = getPhraseEmbedding(question, w2v_model)

retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, df, cleaned_sentences)

In [ ]:
#With BOW, word2Vec with stopwords removed
question_new = "Is it required to have background in algorithms and complexity for data scientist roles"

 
question = clean_sentence(question_new, stopwords=True)
clean_sentences = get_cleaned_sentences(df, stopwords=True)



In [ ]:

#Try BOW
question_embedding = dictionary.doc2bow(question.lower().split())
print("\n\n With BOW\n\n")
retrieveAndPrintFAQAnswer(question_embedding, bow_corpus, df,clean_sentences )

In [ ]:

sent_embeddings = []
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent, w2v_model))

 
question_embedding = getPhraseEmbedding(question, w2v_model)

 

print("\n With Word2Vec\n\n")
retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, df, cleaned_sentences)

 
question = clean_sentence(question_new, stopwords=False)
cleaned_sentences = get_cleaned_sentences(df, stopwords=False)